In [ ]:
# === CONFIGURAÇÃO DA API OPENROUTER (OPCIONAL) ===
# Monitor LLM para otimização inteligente do treinamento

import os
import getpass

# Configuração da API OpenRouter
ENABLE_LLM_MONITOR = True # @param {type:"boolean"}

if ENABLE_LLM_MONITOR:
    print("🤖 Configurando Monitor LLM com OpenRouter...")
    print("📋 O monitor LLM analisa o progresso do treinamento e sugere ajustes automáticos")
    print("💡 Opcional: deixe em branco para desabilitar")
    
    # Input da API key (campo seguro)
    OPENROUTER_API_KEY = getpass.getpass("🔑 Digite sua API Key do OpenRouter (ou Enter para pular): ")
    
    if OPENROUTER_API_KEY and OPENROUTER_API_KEY.strip():
        # Configurar variável de ambiente
        os.environ['OPENROUTER_API_KEY'] = OPENROUTER_API_KEY.strip()
        
        # Testar conexão
        try:
            import requests
            test_url = "https://openrouter.ai/api/v1/models"
            headers = {
                "Authorization": f"Bearer {OPENROUTER_API_KEY.strip()}",
                "Content-Type": "application/json"
            }
            
            response = requests.get(test_url, headers=headers, timeout=10)
            if response.status_code == 200:
                print("✅ API Key válida - Monitor LLM habilitado")
                print("📊 Funcionalidades ativas:")
                print("   - Análise automática a cada 10 épocas")
                print("   - Sugestões de ajuste de learning rate")
                print("   - Detecção de problemas de convergência")
                print("   - Relatórios detalhados de progresso")
                
                # Mostrar modelos disponíveis
                models = response.json().get('data', [])
                claude_models = [m for m in models if 'claude' in m.get('id', '').lower()]
                if claude_models:
                    print(f"🧠 Modelo recomendado: {claude_models[0].get('id', 'claude-3-5-sonnet')}")
            else:
                print(f"⚠️ Erro na validação da API: {response.status_code}")
                print("🔧 Monitor LLM será desabilitado para este treinamento")
                ENABLE_LLM_MONITOR = False
                
        except Exception as e:
            print(f"⚠️ Erro ao testar API: {e}")
            print("🔧 Monitor LLM será desabilitado para este treinamento")
            ENABLE_LLM_MONITOR = False
    else:
        print("ℹ️ API Key não fornecida - Monitor LLM desabilitado")
        ENABLE_LLM_MONITOR = False
else:
    print("🔧 Monitor LLM desabilitado por configuração")
    ENABLE_LLM_MONITOR = False

# Salvar configuração para uso posterior
LLM_CONFIG = {
    'enabled': ENABLE_LLM_MONITOR,
    'provider': 'openrouter',
    'model': 'anthropic/claude-3-5-sonnet-20241022',
    'monitor_every_epochs': 10,
    'api_key_configured': bool(os.environ.get('OPENROUTER_API_KEY'))
}

print(f"\n🎯 Configuração LLM: {'✅ Ativo' if ENABLE_LLM_MONITOR else '❌ Desabilitado'}")

if not ENABLE_LLM_MONITOR:
    print("\n💡 Para habilitar o Monitor LLM:")
    print("1. Crie uma conta em https://openrouter.ai")
    print("2. Obtenha sua API Key")
    print("3. Execute esta célula novamente e forneça a key")
    print("4. O monitor ajudará a otimizar seu treinamento automaticamente")

In [ ]:
# === CONFIGURAÇÃO DO MODO DE TREINAMENTO ===
# Configuração otimizada para A100 40GB VRAM
DEBUG_MODE = False # @param {type:"boolean"}
USE_DRIVE = True # @param {type:"boolean"}
MOUNT_DRIVE = True # @param {type:"boolean"}

# Detectar GPU automaticamente e ajustar configuração
import subprocess
def get_gpu_info():
    try:
        result = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader,nounits'], 
                              capture_output=True, text=True)
        if result.returncode == 0:
            gpu_info = result.stdout.strip().split('\n')[0]
            gpu_name, gpu_memory = gpu_info.split(', ')
            return gpu_name.strip(), int(gpu_memory.strip())
    except:
        pass
    return "Unknown", 0

gpu_name, gpu_memory = get_gpu_info()
print(f"🎮 GPU Detectada: {gpu_name}")
print(f"💾 VRAM: {gpu_memory} MB")

# Configuração automática baseada na GPU
if "A100" in gpu_name and gpu_memory >= 40000:
    print("🚀 MODO A100: Configuração de alta performance")
    BATCH_SIZE = 32
    CONFIG_NAME = "vits2_english_a100_optimized"
    EXPECTED_PERFORMANCE = "5-8 it/s"
    ESTIMATED_TIME = "4-6 horas"
elif "V100" in gpu_name:
    print("⚡ MODO V100: Configuração otimizada")
    BATCH_SIZE = 24
    CONFIG_NAME = "vits2_english_production"
    EXPECTED_PERFORMANCE = "3-5 it/s"
    ESTIMATED_TIME = "6-8 horas"
elif "T4" in gpu_name:
    print("📱 MODO T4: Configuração conservativa")
    BATCH_SIZE = 16
    CONFIG_NAME = "vits2_english_production"
    EXPECTED_PERFORMANCE = "1-2 it/s"
    ESTIMATED_TIME = "12-15 horas"
else:
    print("⚠️ GPU não detectada, usando configuração padrão")
    BATCH_SIZE = 16
    CONFIG_NAME = "vits2_english_production"
    EXPECTED_PERFORMANCE = "1-2 it/s"
    ESTIMATED_TIME = "12-15 horas"

# Configuração do treinamento
if DEBUG_MODE:
    print("🐛 MODO DEBUG: Treinamento rápido para teste")
    EPOCHS = 3
    MAX_SAMPLES = 100
    CONFIG_NAME = "vits2_english_debug"
    BATCH_SIZE = min(BATCH_SIZE, 8)  # Reduzido para debug
else:
    print("🚀 MODO PRODUÇÃO: Treinamento completo")
    EPOCHS = 200
    MAX_SAMPLES = None

print(f"📊 Configuração: {CONFIG_NAME}")
print(f"🔄 Épocas: {EPOCHS}")
print(f"📦 Batch Size: {BATCH_SIZE}")
print(f"📈 Amostras: {MAX_SAMPLES if MAX_SAMPLES else 'Todas (22.910)'}")
print(f"⚡ Performance esperada: {EXPECTED_PERFORMANCE}")
print(f"⏱️ Tempo estimado: {ESTIMATED_TIME}")

In [ ]:
# === CONFIGURAÇÃO DO SISTEMA E CLONAGEM ===
import subprocess
import sys
import os

def run_command(cmd, description):
    """Executa comando com output em tempo real."""
    print(f"🔄 {description}")
    process = subprocess.Popen(
        cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        universal_newlines=True, bufsize=1
    )

    for line in process.stdout:
        print(line.rstrip())

    process.wait()
    if process.returncode != 0:
        raise RuntimeError(f"Comando falhou: {cmd}")
    print(f"✅ {description} - Concluído\n")

# Montar Google Drive se necessário
if MOUNT_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')

    if USE_DRIVE:
        drive_path = '/content/drive/MyDrive/ValeTTS'
        os.makedirs(drive_path, exist_ok=True)
        os.chdir(drive_path)
        print(f"📁 Diretório de trabalho: {drive_path}")

# Verificar GPU
run_command("nvidia-smi", "Verificando GPU disponível")

# Clonar repositório
if not os.path.exists('ValeTTS'):
    run_command(
        "git clone https://github.com/wallaceblaia/ValeTTS-Colab.git ValeTTS",
        "Clonando repositório ValeTTS"
    )
else:
    print("📁 Repositório já existe")

os.chdir('ValeTTS')
run_command("git pull origin main", "Atualizando repositório")
print(f"📍 Diretório atual: {os.getcwd()}")


In [ ]:
# === INSTALAÇÃO DE DEPENDÊNCIAS ESPECÍFICAS PARA INGLÊS ===

print("🔧 Instalando dependências do sistema para inglês...")

# Instalar dependências do sistema para inglês
system_deps = [
    "apt-get update",
    "apt-get install -y espeak espeak-data libespeak1 libespeak-dev",
    "apt-get install -y ffmpeg sox libsox-fmt-all",
    "apt-get install -y language-pack-en",  # Suporte específico para inglês
]

for cmd in system_deps:
    run_command(cmd, f"Sistema: {cmd.split()[-1]}")

print("📦 Instalando dependências Python...")

# Instalar phonemizer específico para inglês
python_deps = [
    "pip install phonemizer==3.3.0",  # Versão específica que funciona bem
    "pip install espeak-python",      # Wrapper para espeak
    "pip install num2words",          # Conversão números para inglês
    "pip install inflect",            # Pluralização e ordinais em inglês
]

for cmd in python_deps:
    run_command(cmd, f"Python: {cmd.split()[-1]}")

# Instalar ValeTTS em modo desenvolvimento
run_command("pip install -e .", "Instalando ValeTTS em modo desenvolvimento")

# Verificar instalações críticas
print("🧪 Verificando instalações...")

verification_commands = [
    ("python -c 'import phonemizer; print(f\"✅ Phonemizer: {phonemizer.__version__}\")'", "Phonemizer"),
    ("python -c 'import espeak; print(\"✅ ESpeak Python wrapper instalado\")'", "ESpeak wrapper"),
    ("espeak --version", "ESpeak sistema"),
    ("python -c 'import valetts; print(f\"✅ ValeTTS instalado\")'", "ValeTTS"),
    ("python -c 'from valetts.data.preprocessing.text_en import EnglishTextPreprocessor; print(\"✅ EnglishTextPreprocessor disponível\")'", "Processador inglês"),
]

for cmd, desc in verification_commands:
    try:
        run_command(cmd, f"Verificar {desc}")
    except Exception as e:
        print(f"⚠️ Aviso: {desc} - {e}")

# Testar processamento básico de inglês
print("🧪 Testando processamento de inglês...")
test_code = '''
from valetts.data.preprocessing.text_en import EnglishTextPreprocessor

# Testar processador de inglês
processor = EnglishTextPreprocessor(
    language="en-us",
    use_phonemes=True,
    normalize_numbers=True
)

# Teste básico
test_text = "Hello world! This is a test with numbers 123 and Mr. Smith."
normalized = processor.normalize_text(test_text)
print(f"✅ Texto normalizado: {normalized}")

# Teste phonemes
try:
    phonemes = processor.text_to_phonemes("hello world")
    print(f"✅ Phonemes: {phonemes}")
except Exception as e:
    print(f"⚠️ Phonemes: {e}")

print("✅ Processador de inglês funcionando!")
'''

try:
    with open("test_english.py", "w") as f:
        f.write(test_code)
    run_command("python test_english.py", "Teste processador inglês")
    run_command("rm test_english.py", "Limpeza")
except Exception as e:
    print(f"⚠️ Teste de inglês falhou: {e}")

print("✅ Todas as dependências para inglês instaladas!")
print("🎯 Sistema pronto para treinamento VITS2 em inglês")

In [ ]:
# === DOWNLOAD DO DATASET ===
dataset_path = "data/generated/Dataset-Unificado"
if not os.path.exists(dataset_path):
    print("📥 Dataset não encontrado localmente")
    print("💡 Certifique-se de ter o dataset disponível em:")
    print("   - Google Drive: /content/drive/MyDrive/ValeTTS/data/generated/Dataset-Unificado")
    print("   - Ou faça upload manual do dataset")

    # Verificar se existe no Drive
    drive_dataset = "/content/drive/MyDrive/ValeTTS/data/generated/Dataset-Unificado"
    if os.path.exists(drive_dataset):
        print(f"✅ Dataset encontrado no Drive: {drive_dataset}")
        print("🔗 Criando link simbólico...")
        os.makedirs("data/generated", exist_ok=True)
        os.symlink(drive_dataset, dataset_path)
    else:
        print("⚠️ Dataset não encontrado!")
        print("📋 Use um dos métodos abaixo:")
        print("1. Upload manual via interface do Colab")
        print("2. Download direto (substitua o link):")
        print("   !gdown --folder 'LINK_DO_GOOGLE_DRIVE_AQUI'")
else:
    print("✅ Dataset já existe")

# Verificar dataset
metadata_file = f"{dataset_path}/metadata.csv"
if os.path.exists(metadata_file):
    import pandas as pd
    df = pd.read_csv(metadata_file)
    print(f"📊 Dataset carregado: {len(df)} amostras")
    print(f"🎙️ Falantes únicos: {df['speaker_id'].nunique()}")
    print(f"🌍 Idiomas: {df['locale'].unique() if 'locale' in df.columns else 'N/A'}")

    # Mostrar amostra dos dados
    print("\n📋 Amostra dos dados:")
    print(df.head(3)[['speaker_id', 'text', 'locale']].to_string())
else:
    print("❌ Arquivo metadata.csv não encontrado!")
    print("💡 Verifique se o dataset foi baixado corretamente")


In [ ]:
# === CONFIGURAÇÃO DINÂMICA DO MODELO PARA INGLÊS ===
import yaml
import pandas as pd

# Primeiro, verificar o dataset
metadata_file = "data/generated/Dataset-Unificado/metadata.csv"
if os.path.exists(metadata_file):
    df = pd.read_csv(metadata_file)
    total_samples = len(df)
    unique_speakers = df['speaker_id'].nunique()
    locales = df['locale'].unique()
    
    print(f"📊 Dataset carregado:")
    print(f"   📈 Total: {total_samples:,} amostras")
    print(f"   🎤 Falantes: {unique_speakers}")
    print(f"   🌍 Idiomas: {locales}")
    
    # Verificar se é inglês
    if 'en' not in locales:
        print("⚠️ AVISO: Dataset não contém inglês!")
        print(f"   Idiomas encontrados: {locales}")
    else:
        english_samples = len(df[df['locale'] == 'en'])
        print(f"   ✅ Amostras em inglês: {english_samples:,}")
else:
    print("❌ Metadata não encontrada!")
    total_samples = 22910  # Fallback
    unique_speakers = 52

# Usar configuração específica para A100 se disponível
if CONFIG_NAME == "vits2_english_a100_optimized":
    # Carregar configuração A100 otimizada
    config_path = f"configs/training/{CONFIG_NAME}.yaml"
    if os.path.exists(config_path):
        with open(config_path, 'r') as f:
            config = yaml.safe_load(f)
        print("✅ Usando configuração A100 otimizada!")
    else:
        print("⚠️ Configuração A100 não encontrada, criando...")
        # Configuração A100 inline
        config = {
            "model": {
                "name": "VITS2",
                "text_encoder_hidden_dim": 512 if not DEBUG_MODE else 256,
                "latent_dim": 512 if not DEBUG_MODE else 256,
                "mel_channels": 80,
                "n_speakers": unique_speakers,
                "speaker_embedding_dim": 1024 if not DEBUG_MODE else 512,
                "generator_initial_channels": 1024 if not DEBUG_MODE else 512,
                "decoder_hidden_dim": 1024 if not DEBUG_MODE else 512,
                "text_processor": "english",
                "inference_only": False
            }
        }
else:
    # Configuração para outras GPUs
    if DEBUG_MODE:
        model_config = {
            "text_encoder_hidden_dim": 128,
            "latent_dim": 128,
            "speaker_embedding_dim": 256,
            "generator_initial_channels": 256,
            "decoder_hidden_dim": 256,
        }
    else:
        model_config = {
            "text_encoder_hidden_dim": 192,
            "latent_dim": 192,
            "speaker_embedding_dim": 512,
            "generator_initial_channels": 512,
            "decoder_hidden_dim": 512,
        }
    
    config = {
        "model": {
            "name": "VITS2",
            "mel_channels": 80,
            "n_speakers": unique_speakers,
            "text_processor": "english",
            "inference_only": False,
            **model_config
        }
    }

# Adicionar configurações de treinamento
config.update({
    "training": {
        "learning_rate": 2.0e-4,
        "batch_size": BATCH_SIZE,
        "max_epochs": EPOCHS,
        "accumulate_grad_batches": 1,
        "max_grad_norm": 1.0,
        "mel_loss_weight": 45.0,
        "kl_loss_weight": 1.0,
        "adv_loss_weight": 1.0,
        "fm_loss_weight": 2.0,
        "duration_loss_weight": 1.0,
        "use_amp": True,
        "gradient_clip_val": 1.0,
        "discriminator_update_frequency": 1,
        "scheduler": {
            "name": "ReduceLROnPlateau",
            "mode": "min",
            "factor": 0.5,
            "patience": 15 if not DEBUG_MODE else 5,
            "min_lr": 1.0e-6
        }
    },
    "data": {
        "dataset_format": "valetts",
        "data_dir": "data/generated/Dataset-Unificado",
        "metadata_file": "data/generated/Dataset-Unificado/metadata.csv",
        "language": "en",  # CRÍTICO: usar "en" para inglês
        "locale_column": "locale",
        "text_processor": {
            "class": "EnglishTextPreprocessor",
            "use_phonemes": True,
            "normalize_numbers": True,
            "normalize_whitespace": True,
            "lowercase": True,
            "backend": "espeak",
            "language": "en-us"
        },
        "sample_rate": 22050,
        "n_mels": 80,
        "n_fft": 1024,
        "hop_length": 256,
        "win_length": 1024,
        "num_workers": 8 if "A100" in gpu_name else 4,
        "pin_memory": True,
        "persistent_workers": True,
        "use_augmentation": True,
        "volume_range": [0.9, 1.1],
        "pitch_range": [-1, 1]
    },
    "logging": {
        "log_dir": "logs",
        "experiment_name": f"vits2_english_{CONFIG_NAME.split('_')[-1]}",
        "checkpoint": {
            "dirpath": f"checkpoints/vits2_english_{CONFIG_NAME.split('_')[-1]}",
            "filename": f"vits2_english-{{epoch:03d}}-{{epoch/val_loss_total:.3f}}",
            "monitor": "epoch/val_loss_total",
            "mode": "min",
            "save_top_k": 3 if DEBUG_MODE else 5,
            "save_last": True,
            "every_n_epochs": 1 if DEBUG_MODE else 10
        },
        "early_stopping": {
            "monitor": "epoch/val_loss_total",
            "mode": "min",
            "patience": 10 if DEBUG_MODE else 30,
            "min_delta": 0.001
        },
        "tensorboard": {
            "save_dir": "logs/tensorboard",
            "name": f"vits2_english_{CONFIG_NAME.split('_')[-1]}"
        }
    },
    "hardware": {
        "accelerator": "gpu",
        "devices": 1,
        "precision": "16-mixed",
        "strategy": "auto",
        "benchmark": True if "A100" in gpu_name else False
    },
    "validation": {
        "val_check_interval": 1.0,
        "generate_samples": True,
        "sample_every_n_epochs": 1 if DEBUG_MODE else 10,
        "limit_val_batches": 1.0
    },
    "dataset_config": {
        "expected_locale": "en",  # CRÍTICO: valor correto para inglês
        "validate_files": True,
        "cache_preprocessing": True,
        "audio_column": "audio_path",
        "text_column": "text_normalized",
        "speaker_column": "speaker_id",
        "locale_column": "locale"
    },
    # CONFIGURAÇÃO LLM INTEGRADA
    "llm_monitor": LLM_CONFIG
})

# Adicionar limitação de amostras para debug
if DEBUG_MODE:
    config["data"]["max_samples_debug"] = MAX_SAMPLES

# Criar diretórios necessários
os.makedirs("configs/training", exist_ok=True)
os.makedirs(config["logging"]["checkpoint"]["dirpath"], exist_ok=True)
os.makedirs("logs/tensorboard", exist_ok=True)

# Salvar configuração
config_path = f"configs/training/{CONFIG_NAME}.yaml"
with open(config_path, 'w') as f:
    yaml.dump(config, f, default_flow_style=False, indent=2)

print(f"✅ Configuração criada: {config_path}")
print(f"🎯 Modo: {CONFIG_NAME.upper()}")
print(f"📊 Épocas: {EPOCHS}")
print(f"📦 Batch Size: {BATCH_SIZE}")
print(f"🎤 Falantes: {config['model']['n_speakers']}")
print(f"🌍 Idioma: INGLÊS (locale='en')")
print(f"🤖 Monitor LLM: {'✅ Ativo' if LLM_CONFIG['enabled'] else '❌ Desabilitado'}")
print(f"💾 Dimensões do modelo:")
print(f"   - Hidden: {config['model']['text_encoder_hidden_dim']}")
print(f"   - Speaker: {config['model']['speaker_embedding_dim']}")
print(f"   - Generator: {config['model']['generator_initial_channels']}")
print(f"   - Decoder: {config['model']['decoder_hidden_dim']}")

# Verificar compatibilidade de dimensões CRÍTICA
if config['model']['speaker_embedding_dim'] == config['model']['decoder_hidden_dim']:
    print("✅ Dimensões compatíveis - Sem erro de tensor!")
else:
    print("❌ AVISO: Dimensões incompatíveis detectadas!")
    print(f"   Speaker: {config['model']['speaker_embedding_dim']}")
    print(f"   Decoder: {config['model']['decoder_hidden_dim']}")

print(f"\n🚀 Configuração otimizada para {gpu_name} pronta!")

if LLM_CONFIG['enabled']:
    print("\n🤖 Monitor LLM ativo - benefícios:")
    print("   📊 Análise inteligente do progresso")
    print("   ⚙️ Ajustes automáticos de hiperparâmetros")
    print("   🎯 Detecção precoce de problemas")
    print("   📈 Relatórios detalhados de treinamento")

In [ ]:
# === INICIAR TREINAMENTO OTIMIZADO PARA INGLÊS ===
print(f"🚀 Iniciando treinamento VITS2 - Modo: {CONFIG_NAME.upper()}")
print(f"📁 Configuração: {config_path}")
print(f"🎮 GPU: {gpu_name} ({gpu_memory} MB)")
print(f"📦 Batch Size: {BATCH_SIZE}")
print(f"⏱️ Estimativa: {ESTIMATED_TIME}")
print(f"⚡ Performance esperada: {EXPECTED_PERFORMANCE}")
print(f"🤖 Monitor LLM: {'✅ Ativo' if LLM_CONFIG['enabled'] else '❌ Desabilitado'}")
print("\n" + "="*60)

# Verificar se temos script específico para inglês
english_script = "scripts/train_vits2_english.py"
if os.path.exists(english_script):
    print("✅ Usando script específico para inglês")
    base_cmd = f"python {english_script} --config {config_path}"
else:
    print("⚠️ Script específico não encontrado, usando script padrão")
    base_cmd = f"python scripts/train_vits2.py --config {config_path}"

# Configurar comando baseado no LLM
if LLM_CONFIG['enabled']:
    print("🤖 Monitor LLM ativo - treinamento inteligente habilitado")
    cmd = base_cmd  # LLM já está na configuração
else:
    print("🔧 Monitor LLM desabilitado - usando modo padrão")
    cmd = f"{base_cmd} --disable-llm"

# Configurar CUDA para otimização máxima
os.environ['CUDA_LAUNCH_BLOCKING'] = '0'  # Async CUDA
os.environ['TORCH_CUDNN_V8_API_ENABLED'] = '1'  # CuDNN v8
os.environ['TORCH_ALLOW_TF32_CUBLAS_OVERRIDE'] = '1'  # TF32 para A100

if "A100" in gpu_name:
    print("🚀 Configurações especiais para A100:")
    print("   - TF32 habilitado para máxima performance")
    print("   - CuDNN v8 API ativado")
    print("   - Tensor cores ativados automaticamente")
    os.environ['TORCH_ALLOW_TF32'] = '1'
    os.environ['TORCH_CUDNN_ALLOW_TF32'] = '1'

# Configurações específicas para Monitor LLM
if LLM_CONFIG['enabled']:
    print("\n🤖 Configurações do Monitor LLM:")
    print(f"   🧠 Modelo: {LLM_CONFIG['model']}")
    print(f"   📊 Análise a cada: {LLM_CONFIG['monitor_every_epochs']} épocas")
    print(f"   🔄 Provider: {LLM_CONFIG['provider']}")
    print("   📈 Funcionalidades ativas:")
    print("      - Otimização automática de learning rate")
    print("      - Detecção de problemas de convergência")
    print("      - Sugestões de ajustes de hiperparâmetros")
    print("      - Relatórios detalhados de progresso")

# Executar treinamento com output em tempo real
print(f"\n📝 Comando: {cmd}")
print("🏁 Iniciando treinamento...\n")

try:
    # Executar treinamento
    run_command(cmd, f"Treinamento VITS2 {CONFIG_NAME.upper()}")
    
    print("\n" + "="*60)
    print("🎉 TREINAMENTO CONCLUÍDO COM SUCESSO!")
    
    # Verificar checkpoints gerados
    checkpoint_dir = config["logging"]["checkpoint"]["dirpath"]
    if os.path.exists(checkpoint_dir):
        checkpoints = [f for f in os.listdir(checkpoint_dir) if f.endswith('.ckpt')]
        print(f"📁 Checkpoints gerados: {len(checkpoints)}")
        
        if checkpoints:
            # Mostrar checkpoints mais recentes
            checkpoint_paths = [os.path.join(checkpoint_dir, f) for f in checkpoints]
            checkpoint_paths.sort(key=os.path.getmtime, reverse=True)
            
            print("📦 Checkpoints disponíveis:")
            for i, ckpt in enumerate(checkpoint_paths[:3]):  # Mostrar 3 mais recentes
                size_mb = os.path.getsize(ckpt) / 1024 / 1024
                print(f"   {i+1}. {os.path.basename(ckpt)} ({size_mb:.1f} MB)")
    
    # Verificar logs do TensorBoard
    tensorboard_dir = config["logging"]["tensorboard"]["save_dir"]
    if os.path.exists(tensorboard_dir):
        print(f"📊 Logs TensorBoard: {tensorboard_dir}")
    
    # Verificar logs do Monitor LLM se ativo
    if LLM_CONFIG['enabled']:
        llm_logs_dir = "logs/llm_monitor"
        if os.path.exists(llm_logs_dir):
            llm_files = [f for f in os.listdir(llm_logs_dir) if f.endswith('.json')]
            print(f"🤖 Análises LLM geradas: {len(llm_files)}")
            if llm_files:
                latest_analysis = sorted(llm_files)[-1]
                print(f"   📄 Última análise: {latest_analysis}")
    
    print(f"\n✅ Modelo treinado com sucesso para inglês!")
    print(f"🎯 Configuração: {CONFIG_NAME}")
    print(f"🌍 Idioma: Inglês")
    print(f"📈 Dataset: 22.910 amostras")
    print(f"🤖 Monitor LLM: {'Usado' if LLM_CONFIG['enabled'] else 'Não usado'}")

except Exception as e:
    print(f"\n❌ ERRO DURANTE TREINAMENTO:")
    print(f"   {str(e)}")
    print(f"\n🔍 Diagnóstico:")
    
    # Verificar se arquivos necessários existem
    required_files = [
        config_path,
        config["data"]["metadata_file"],
        config["data"]["data_dir"]
    ]
    
    for file_path in required_files:
        if os.path.exists(file_path):
            print(f"   ✅ {file_path}")
        else:
            print(f"   ❌ {file_path} - NÃO ENCONTRADO")
    
    # Verificar GPU
    if torch.cuda.is_available():
        print(f"   ✅ CUDA disponível: {torch.cuda.get_device_name(0)}")
    else:
        print("   ❌ CUDA não disponível")
    
    # Verificar configuração LLM se ativa
    if LLM_CONFIG['enabled']:
        if os.environ.get('OPENROUTER_API_KEY'):
            print("   ✅ API Key OpenRouter configurada")
        else:
            print("   ❌ API Key OpenRouter não encontrada")
    
    raise

In [ ]:
# === DOWNLOAD DOS RESULTADOS ===
import zipfile
from datetime import datetime
import glob

# Criar arquivo ZIP com resultados
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
zip_filename = f"valetts_vits2_english_{CONFIG_NAME.split('_')[-1]}_{timestamp}.zip"

print(f"📦 Criando arquivo ZIP: {zip_filename}")

with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    # Adicionar checkpoints
    checkpoint_dir = config["logging"]["checkpoint"]["dirpath"]
    if os.path.exists(checkpoint_dir):
        for root, dirs, files in os.walk(checkpoint_dir):
            for file in files:
                if file.endswith('.ckpt'):
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, os.path.relpath(file_path, '.'))

    # Adicionar configuração
    zipf.write(config_path, config_path)

    # Adicionar amostras geradas
    if os.path.exists("samples"):
        for root, dirs, files in os.walk("samples"):
            for file in files:
                if file.endswith('.wav'):
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, os.path.relpath(file_path, '.'))

    # Adicionar logs principais
    if os.path.exists("logs/training.log"):
        zipf.write("logs/training.log", "logs/training.log")

print(f"✅ Arquivo criado: {zip_filename}")
print(f"💾 Tamanho: {os.path.getsize(zip_filename) / 1024 / 1024:.1f} MB")

# Download no Colab
if 'google.colab' in sys.modules:
    from google.colab import files
    print("⬇️ Iniciando download...")
    files.download(zip_filename)
    print("✅ Download concluído!")

print("\n🎯 TREINAMENTO FINALIZADO!")
print(f"📊 Modo: {CONFIG_NAME.upper()}")
print(f"⏱️ Status: {'Teste concluído' if DEBUG_MODE else 'Produção concluída'}")
print(f"📦 Resultados salvos em: {zip_filename}")

# Encontrar e mostrar checkpoints disponíveis
checkpoint_dir = config["logging"]["checkpoint"]["dirpath"]
checkpoints = glob.glob(f"{checkpoint_dir}/*.ckpt")
if checkpoints:
    print(f"\n📁 Checkpoints disponíveis ({len(checkpoints)}):")
    for ckpt in sorted(checkpoints)[-3:]:  # Mostrar últimos 3
        size_mb = os.path.getsize(ckpt) / 1024 / 1024
        print(f"   📦 {os.path.basename(ckpt)} ({size_mb:.1f} MB)")
else:
    print("\n⚠️ Nenhum checkpoint encontrado")
